### 데이터 분석

In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from collections import Counter
from konlpy.corpus import kobill
from konlpy.tag import Okt
import urllib.request
from tqdm import tqdm
from konlpy.corpus import kolaw
from konlpy.utils import pprint
from konlpy.tag import Okt
from konlpy.tag import Komoran
from sklearn.model_selection import train_test_split

In [2]:
# Get current working directory
cwd = os.getcwd()

# Print current working directory
print("현재 작업 디렉토리:", cwd)


현재 작업 디렉토리: c:\Users\user1\Desktop\공부자료\제2회 ETRI 휴먼이해 인공지능 논문경진대회\감정분류(학습,평가,예측)


In [3]:
#파일 불러오기
data = pd.read_csv('./KEMDy20_v1 감정분류 데이터/txt.csv', header = 0)
data.head(10)

,Text,Emotion,Arousal,Valence
0,아 친구들도?,neutral,2.8,3.8
1,나는 생일?,neutral,3.1,3.3
2,생일날이면은 내가 고기를 되게 좋아하니까 엄마도 그걸 아니까 미역국도 같이 해주시...,neutral,3.1,3.3
3,선물이라 이 보통 돈으로 주시지.,happy,3.1,3.6
4,용돈으로 주시고.,neutral,3.2,3.5
5,옛날에는 내가 원하는 게 뭔지 일단 물어보시곤 했던 거 같애.,neutral,2.9,3.0
6,옷이라든가 갖고 싶은 거 있나라든가 근데 이제 나이를 먹을수록 현금으로 챙겨주...,neutral,3.2,3.6
7,응 나도 되게 여행 가는 걸 좋아하긴 하는데 가족 다 같이 가는 것보다 되게 엄마...,neutral,3.1,3.1
8,어 단둘이.,neutral,2.7,3.0
9,그 되게 아빠나 오빠가 시간이 안 나거나 꼭 어디 약속이 있다거나 그래가지고 엄마...,neutral,2.7,2.8


In [4]:
print('학습데이터 전체 개수: {}'.format(len(data)))

학습데이터 전체 개수: 13462


In [5]:
#리뷰 전체길이 확인
data_length = data['Text'].astype(str).apply(len)
data_length.head(10)

0      9
1      7
2    107
3     20
4     10
5     36
6     51
7     66
8      6
9     75
Name: Text, dtype: int64

In [6]:
#리뷰 통계 정보
print('리뷰 길이 최댓값: {}'.format(np.max(data_length)))
print('리뷰 길이 최솟값: {}'.format(np.min(data_length)))
print('리뷰 길이 평균값: {:.2f}'.format(np.mean(data_length)))
print('리뷰 길이 표준편차: {:.2f}'.format(np.std(data_length)))
print('리뷰 길이 중간값: {}'.format(np.median(data_length)))
print('리뷰 길이 제1사분위: {}'.format(np.percentile(data_length,25)))
print('리뷰 길이 제3사분위: {}'.format(np.percentile(data_length,75)))

리뷰 길이 최댓값: 320
리뷰 길이 최솟값: 5
리뷰 길이 평균값: 35.75
리뷰 길이 표준편차: 29.53
리뷰 길이 중간값: 26.0
리뷰 길이 제1사분위: 15.0
리뷰 길이 제3사분위: 47.0


In [7]:
# 문자열 아닌 데이터 모두 제거
data_text = [txt for txt in data['Text'] if type(txt) is str]
data_text

[' 아 친구들도? ',
 ' 나는 생일?',
 '생일날이면은  내가 고기를 되게 좋아하니까 엄마도 그걸 아니까 미역국도 같이 해주시는데 꼭 고기를 같이 구워주시거나 아니면 여행을 가서 고기를 구워 먹는다거나 꼭 고깃집에 가서 어 해먹어 꼭.',
 ' 선물이라 이 보통 돈으로 주시지. ',
 ' 용돈으로 주시고.',
 '옛날에는   내가 원하는 게 뭔지 일단 물어보시곤 했던 거 같애.',
 ' 옷이라든가 갖고 싶은 거 있나라든가  근데 이제  나이를 먹을수록 현금으로 챙겨주시더라구.',
 '응 나도 되게  여행 가는 걸 좋아하긴 하는데 가족 다 같이 가는 것보다 되게 엄마랑 여행 가는 게 많았던 거 같아. ',
 '어 단둘이.',
 '그 되게 아빠나 오빠가 시간이 안 나거나 꼭  어디 약속이 있다거나 그래가지고 엄마가 주말마다 좀 같이 둘이 어디 데려다가 줬어 그냥.',
 '여행 여행을.',
 ' 최근 최근에?',
 ' 굳이 가자면 나는 부산에 가면 꼭 엄마가 생각나.',
 '어 부산을 좀 가기도 많이 갔는데 부산에 엄마가 좀  그 친척들이  가지고.',
 '아  되게 되게 처음이라 엉망진창이었던 거 같긴 한데.',
 '나는 어 어머니 엄마가 이제 생일 때마다 이제 미역국도 많이 해서 미역국을 되게 좋아하거든.',
 '전에도 말했지만 그래서 미역국이랑 볶음김치 막 해주셔서 맨날 그렇게 차려주시는데 어 옛날에는 이제 집에서 밥 먹는 거 보다 그 캔모아 빙수 있는데 있잖아.',
 ' 어 응 거기를 되게 엄 집 앞에 그 조그만한 가게로 있었는데 거기를 엄청 생일 때마다 매번 같이 데려갔었거든.',
 '거기서 엄마랑 둘이 생일 파티 그냥 간단하게 하는 경우도 있었고  보 음 보통 그렇게 아니면 친구들 이제 거기 빙수 가게로 불러서 같이 빙수 먹으면서 케익도 불고 막 이랬었었어.',
 '응응 그래서 생일 때 그렇게 했던 경험이 있는데 그때가 이제 초등학교 때였는데 이제 중학교 때쯤 가게가 문을 닫았거든 집 앞이었는데.',
 '그래서 이제 막 지역 돌아다니면서 캔모아 빙

In [8]:
okt = Okt()
komoran = Komoran()

In [93]:
# Tokenize and remove stop words
tokens = komoran.nouns(' '.join(data_text))
filtered_tokens = [x for x in tokens if len(x) > 1]

# Generate word cloud
wordcloud = WordCloud('./nsmc/NanumSquare_acB.ttf').generate(' '.join(filtered_tokens))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()


In [9]:
data['Emotion'].value_counts()

neutral                           11120
happy                              1528
angry                               227
surprise                            212
sad                                 192
disqust                             101
fear                                 60
happy;surprise                        8
angry;disqust                         3
happy;fear                            2
angry;neutral;disqust                 2
neutral;disqust;sad                   1
angry;neutral;disqust;fear;sad        1
happy;sad                             1
happy;surprise;neutral                1
happy;angry;neutral                   1
happy;neutral;disqust                 1
happy;neutral;fear                    1
Name: Emotion, dtype: int64

In [10]:
# 7가지 감정만 남기기
valid_emotions = ['neutral', 'happy', 'angry', 'surprise', 'sad']
data = data[data['Emotion'].isin(valid_emotions)]
data['Emotion'].value_counts()

neutral     11120
happy        1528
angry         227
surprise      212
sad           192
Name: Emotion, dtype: int64

In [11]:
# create a dictionary to map emotions to numbers
emotion_dict = {
    'neutral': 0,
    'happy': 1,
    'angry': 2,
    'surprise': 3,
    'sad': 4
}

# use the replace function to replace the values in the "Emotion" column with numbers
data['Emotion'] = data['Emotion'].replace(emotion_dict)

In [12]:
# Group by emotion category
grouped = data.groupby('Emotion')

# Create a new empty dataframe
new_data = pd.DataFrame(columns=['Text', 'Emotion', 'Arousal', 'Valence'])

# Loop through each emotion category and extract 200 instances randomly
for emotion, group in grouped:
    if len(group) > 200:
        group = group.sample(n=200, random_state=42)
    new_data = new_data.append(group, ignore_index=True)

# Shuffle the new dataframe
new_data = new_data.sample(frac=1, random_state=42)

# Take the first 1000 instances
data = new_data.iloc[:1000, :]

data['Emotion'].value_counts()

C:\Users\user1\AppData\Local\Temp\ipykernel_28100\1249974624.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_data = new_data.append(group, ignore_index=True)


1    200
2    200
0    200
3    200
4    192
Name: Emotion, dtype: int64

In [13]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [14]:
train_data.head()

,Text,Emotion,Arousal,Valence
192,그니까 이제 저는 간부로 생활하다 보니깐 이제 사실 이제 진짜 끈 꼰대의 끝판왕은 ...,0,3.6,3.3
695,아 그런 사람들이 회사까지 찾아오는 거예요?,3,3.5,3.3
318,뭐 빅스비라던지.,1,3.7,4.1
122,저는 솔직히 될 수 있다고 생각을 해요.,0,3.2,3.1
2,음 착한 착한 척 좀 했네 음.,0,3.0,3.3


In [15]:
test_data.head()

,Text,Emotion,Arousal,Valence
31,거기는 애기들이 너무 아빠를 대면 대면해 해가지고 어 얘기 많았어.,0,3.3,2.4
361,아 애들이 있으니까 따라 하는구나.,1,3.5,3.7
15,진짜 좋아하나 보다 유재석을.,0,3.2,3.8
722,어머님이?,3,3.3,3.2
363,너 아니라고?,1,3.3,3.6


### 데이터 전처리

In [16]:
import numpy as np
import pandas as pd
import re
import json
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

data = pd.read_csv('./KEMDy20_v1 감정분류 데이터/txt.csv', header = 0)

data['Text'][:5]

0                                             아 친구들도? 
1                                               나는 생일?
2    생일날이면은  내가 고기를 되게 좋아하니까 엄마도 그걸 아니까 미역국도 같이 해주시...
3                                  선물이라 이 보통 돈으로 주시지. 
4                                            용돈으로 주시고.
Name: Text, dtype: object

##### 전처리를 도와주는 함수 만들기

In [17]:
#전처리 함수 만들기
def preprocessing(review, okt, remove_stopwords = False, stop_words =[]):
  #함수인자설명
  # review: 전처리할 텍스트
  # okt: okt객체를 반복적으로 생성하지 않고 미리 생성 후 인자로 받음
  # remove_stopword: 불용어를 제거할지 여부 선택. 기본값 False
  # stop_words: 불용어 사전은 사용자가 직접 입력, 기본값 빈 리스트

  # 1. 한글 및 공백 제외한 문자 모두 제거
  review_text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]','',review)
  
  #2. okt 객체를 활용하여 형태소 단어로 나눔
  word_review = okt.morphs(review_text,stem=True)

  if remove_stopwords:
    #3. 불용어 제거(선택)
    word_review = [x for x in word_review if len(x) > 1]
  return word_review

#### 전체 학습데이터 및 평가데이터 리뷰 전처리하기

In [18]:
# 전체 텍스트 전처리
okt = Okt()
clean_train_text = []

for review in train_data['Text']:
  # 리뷰가 문자열인 경우만 전처리 진행
  if type(review) == str:
    clean_train_text.append(preprocessing(review,okt,remove_stopwords=True))
  else:
    clean_train_text.append([]) #str이 아닌 행은 빈칸으로 놔두기

In [19]:
#테스트 리뷰도 동일하게 전처리
clean_test_text = []
for review in test_data['Text']:
  if type(review) == str:
    clean_test_text.append(preprocessing(review, okt, remove_stopwords=True))
  else:
    clean_test_text.append([])

#### 문자로 되어있는 리뷰데이터를 인덱스 벡터로 변환

In [20]:
# 인덱스 벡터 변환 후 일정 길이 넘어가거나 모자라는 리뷰 패딩처리
tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_train_text)
train_sequences = tokenizer.texts_to_sequences(clean_train_text)
test_sequences = tokenizer.texts_to_sequences(clean_test_text)

word_vocab = tokenizer.word_index #단어사전형태
MAX_SEQUENCE_LENGTH = 8 #문장 최대 길이

#학습 데이터
train_inputs = pad_sequences(train_sequences,maxlen=MAX_SEQUENCE_LENGTH,padding='post')

#학습 데이터 라벨 벡터화
train_labels = np.array(train_data['Emotion'])

#평가 데이터 
test_inputs = pad_sequences(test_sequences,maxlen=MAX_SEQUENCE_LENGTH,padding='post')
#평가 데이터 라벨 벡터화
test_labels = np.array(test_data['Emotion'])

#### 처리 완료된 데이터 넘파이 파일로 저장

In [21]:
DEFAULT_PATH  = './KEMDy20_v1 감정분류 데이터/' # 경로지정
DATA_PATH = 'CLEAN_DATA/' #.npy파일 저장 경로지정
TRAIN_INPUT_DATA = 'train_input.npy'
TRAIN_LABEL_DATA = 'train_label.npy'
TEST_INPUT_DATA = 'test_input.npy'
TEST_LABEL_DATA = 'test_label.npy'
DATA_CONFIGS = 'data_configs.json'

data_configs={}
data_configs['vocab'] = word_vocab
data_configs['vocab_size'] = len(word_vocab) + 1

#전처리한 데이터들 파일로저장
import os

if not os.path.exists(DEFAULT_PATH + DATA_PATH):
  os.makedirs(DEFAULT_PATH+DATA_PATH)

#전처리 학습데이터 넘파이로 저장
np.save(open(DEFAULT_PATH+DATA_PATH+TRAIN_INPUT_DATA,'wb'),train_inputs)
np.save(open(DEFAULT_PATH+DATA_PATH+TRAIN_LABEL_DATA,'wb'),train_labels)
#전처리 테스트데이터 넘파이로 저장
np.save(open(DEFAULT_PATH+DATA_PATH+TEST_INPUT_DATA,'wb'),test_inputs)
np.save(open(DEFAULT_PATH+DATA_PATH+TEST_LABEL_DATA,'wb'),test_labels)

#데이터 사전 json으로 저장
json.dump(data_configs,open(DEFAULT_PATH + DATA_PATH + DATA_CONFIGS,'w'),ensure_ascii=False)

### 학습하기

#### 학습데이터 및 전처리 데이터 불러오기

In [30]:
# 학습 데이터 불러오기
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import layers
import json

#전처리 데이터 불러오기
DATA_PATH = './KEMDy20_v1 감정분류 데이터/CLEAN_DATA/'
DATA_OUT = './KEMDy20_v1 감정분류 데이터/DATA_OUT/'
INPUT_TRAIN_DATA = 'train_input.npy'
LABEL_TRAIN_DATA = 'train_label.npy'
DATA_CONFIGS = 'data_configs.json'

train_input = np.load(open(DATA_PATH + INPUT_TRAIN_DATA,'rb'))
train_input = pad_sequences(train_input, maxlen=train_input.shape[1])
train_label = np.load(open(DATA_PATH + LABEL_TRAIN_DATA,'rb'), allow_pickle=True)
train_label = train_label.astype('float32')
prepro_configs = json.load(open(DATA_PATH+DATA_CONFIGS,'r'))

#### 파라미터 세팅하기

In [31]:
model_name= 'cnn_classifier_kr'
BATCH_SIZE = 512
NUM_EPOCHS = 20
VALID_SPLIT = 0.1
MAX_LEN = train_input.shape[1]

kargs={'model_name': model_name, 'vocab_size':prepro_configs['vocab_size'],'embedding_size':128, 'num_filters':100,'dropout_rate':0.5, 'hidden_dimension':250,'output_dimension':5}

#### 모델 함수 만들기

In [43]:
class CNNClassifier(tf.keras.Model):

  def __init__(self, **kargs):
    super(CNNClassifier, self).__init__(name=kargs['model_name'])
    self.embedding = layers.Embedding(input_dim=kargs['vocab_size'], output_dim=kargs['embedding_size'])
    self.conv_list = [layers.Conv1D(filters=kargs['num_filters'], kernel_size=kernel_size, padding='valid',activation = tf.keras.activations.relu,
                                    kernel_constraint = tf.keras.constraints.MaxNorm(max_value=3)) for kernel_size in [3,4,5]]
    self.pooling = layers.GlobalMaxPooling1D()
    self.dropout = layers.Dropout(kargs['dropout_rate'])
    self.fc1 = layers.Dense(units=kargs['hidden_dimension'],
                            activation = tf.keras.activations.tanh,
                            kernel_constraint=tf.keras.constraints.MaxNorm(max_value=3.))
    self.fc2 = layers.Dense(units=kargs['output_dimension'],
                            activation=tf.keras.activations.softmax,
                            kernel_constraint= tf.keras.constraints.MaxNorm(max_value=3.))
    

  def call(self,x):
    x = self.embedding(x)
    x = self.dropout(x)
    x = tf.concat([self.pooling(conv(x)) for conv in self.conv_list], axis = 1)
    x = self.fc1(x)
    x = self.fc2(x)
    return x

#### 학습하기

In [44]:
model = CNNClassifier(**kargs)
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss = tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics = [tf.keras.metrics.SparseCategoricalAccuracy()])

#검증 정확도를 통한 EarlyStopping 기능 및 모델 저장 방식 지정
earlystop_callback = EarlyStopping(monitor='val_accuracy', min_delta=0.0001, patience=2)
checkpoint_path = DATA_OUT + model_name +'\weights.h5'
checkpoint_dir = os.path.dirname(checkpoint_path)

if os.path.exists(checkpoint_dir):
  print("{} -- Folder already exists \n".format(checkpoint_dir))
else:
  os.makedirs(checkpoint_dir, exist_ok=True)
  print("{} -- Folder create complete \n".format(checkpoint_dir))

cp_callback = ModelCheckpoint(
    checkpoint_path, monitor = 'val_accuracy', verbose=1, save_best_only = True,
    save_weights_only=True
)

history = model.fit(train_input, train_label, batch_size=BATCH_SIZE, epochs = NUM_EPOCHS,
                    validation_split=VALID_SPLIT, callbacks=[earlystop_callback, cp_callback])

./KEMDy20_v1 감정분류 데이터/DATA_OUT/cnn_classifier_kr -- Folder already exists 

Epoch 1/20
2/2 [==============================] - 1s 368ms/step - loss: 1.6142 - sparse_categorical_accuracy: 0.2118 - val_loss: 1.5932 - val_sparse_categorical_accuracy: 0.2750
Epoch 2/20
2/2 [==============================] - 0s 41ms/step - loss: 1.5647 - sparse_categorical_accuracy: 0.3282 - val_loss: 1.5690 - val_sparse_categorical_accuracy: 0.2875
Epoch 3/20
2/2 [==============================] - 0s 39ms/step - loss: 1.5323 - sparse_categorical_accuracy: 0.3170 - val_loss: 1.5470 - val_sparse_categorical_accuracy: 0.2875
Epoch 4/20
2/2 [==============================] - 0s 39ms/step - loss: 1.5027 - sparse_categorical_accuracy: 0.3268 - val_loss: 1.5330 - val_sparse_categorical_accuracy: 0.3250
Epoch 5/20
2/2 [==============================] - 0s 38ms/step - loss: 1.4772 - sparse_categorical_accuracy: 0.4011 - val_loss: 1.5253 - val_sparse_categorical_accuracy: 0.3250
Epoch 6/20
2/2 [======================

#### 모델저장 pickle버전

In [45]:
import pickle

with open('./KEMDy20_v1 감정분류 데이터/model_save/weights.pkl', 'wb') as f:
    pickle.dump(model, f)

#### 모델저장 h5버전

In [46]:
# import os
# from keras.models import save_model

# # 저장할 디렉토리 경로
# path = 'nsmc/model_save'
# if not os.path.isdir(path):
#     os.makedirs(path)

# # 모델 저장하기
# save_model(model, path+'/weights.h5')


### 평가하기

In [47]:
INPUT_TEST_DATA = 'test_input.npy'
LABEL_TEST_DATA = 'test_label.npy'
SAVE_FILE_NM = 'weights.h5'

test_input = np.load(open(DATA_PATH+INPUT_TEST_DATA,'rb'))
test_input = pad_sequences(test_input,maxlen=test_input.shape[1])
test_label_data = np.load(open(DATA_PATH + LABEL_TEST_DATA, 'rb'), allow_pickle=True)
test_label_data = test_label_data.astype('float32')

# 저장한 모델 pickle로 불러오기
with open('./KEMDy20_v1 감정분류 데이터/model_save/weights.pkl', 'rb') as f:
    model = pickle.load(f)
model.evaluate(test_input, test_label_data)

7/7 [==============================] - 0s 2ms/step - loss: 1.4476 - sparse_categorical_accuracy: 0.4020


[1.4475818872451782, 0.402010053396225]

### 예측하기

In [48]:
import numpy as np
import pandas as pd
import re
import json
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import joblib
from scipy.stats import mode

okt = Okt()
tokenizer  = Tokenizer()

DATA_CONFIGS = 'data_configs.json'
prepro_configs = json.load(open('./KEMDy20_v1 감정분류 데이터/CLEAN_DATA/'+DATA_CONFIGS,'r'))
prepro_configs['vocab'] = word_vocab

tokenizer.fit_on_texts(word_vocab)

MAX_LENGTH = 8 #문장최대길이

sentence = input('감정을 분류할 문장을 입력해 주세요.: ')
sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣//s ]','', sentence)
stopwords = ['은','는','이','가','하','아','것','들','의','있','되','수','보','주','등','한'] # 불용어 추가할 것이 있으면 이곳에 추가
sentence = okt.morphs(sentence, stem=True) # 토큰화
sentence = [word for word in sentence if not word in stopwords] # 불용어 제거
vector  = tokenizer.texts_to_sequences(sentence)
pad_new = pad_sequences(vector, maxlen = MAX_LENGTH) # 패딩

model = joblib.load('./KEMDy20_v1 감정분류 데이터/model_save/weights.pkl') #모델 불러오기
predictions = model.predict(pad_new)
label = np.argmax(predictions, axis=1)
label = mode(label)
label = label.mode[0]

if label == 0:
  print("This is a neutral text.")
elif label == 1:
  print("This is a happy negative text.")
elif label == 2:
  print("This is a angry text.")
elif label == 3:
  print("This is a surprise positive text.")
else:
  print("This is an sad text.")

1/1 [==============================] - 0s 75ms/step
This is a surprise positive text.


C:\Users\user1\AppData\Local\Temp\ipykernel_28100\1823794897.py:33: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  label = mode(label)
